In [4]:
!pip install mxnet #autogluon.tabular

  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 5.4 MB/s eta 0:00:0000:0100:01
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import random
from autogluon.tabular import TabularPredictor
#import mxnet as mx

np.random.seed(123)
random.seed(123)
#mx.random.seed(123)

In [8]:
!mkdir -p product_sentiment_machine_hack
!wget https://autogluon-text-data.s3.amazonaws.com/multimodal_text/machine_hack_product_sentiment/train.csv -O product_sentiment_machine_hack/train.csv
!wget https://autogluon-text-data.s3.amazonaws.com/multimodal_text/machine_hack_product_sentiment/dev.csv -O product_sentiment_machine_hack/dev.csv
!wget https://autogluon-text-data.s3.amazonaws.com/multimodal_text/machine_hack_product_sentiment/test.csv -O product_sentiment_machine_hack/test.csv

--2025-04-01 12:55:06--  https://autogluon-text-data.s3.amazonaws.com/multimodal_text/machine_hack_product_sentiment/train.csv
Resolving autogluon-text-data.s3.amazonaws.com (autogluon-text-data.s3.amazonaws.com)... 52.216.213.57, 52.216.184.179, 3.5.29.97, ...
Connecting to autogluon-text-data.s3.amazonaws.com (autogluon-text-data.s3.amazonaws.com)|52.216.213.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689486 (673K) [text/csv]
Saving to: ‘product_sentiment_machine_hack/train.csv’

product_sentiment_m 100%[===================>] 673.33K  --.-KB/s    in 0.08s   

2025-04-01 12:55:06 (7.77 MB/s) - ‘product_sentiment_machine_hack/train.csv’ saved [689486/689486]

--2025-04-01 12:55:07--  https://autogluon-text-data.s3.amazonaws.com/multimodal_text/machine_hack_product_sentiment/dev.csv
Resolving autogluon-text-data.s3.amazonaws.com (autogluon-text-data.s3.amazonaws.com)... 52.217.1.228, 52.216.184.179, 3.5.29.97, ...
Connecting to autogluon-text-data.s3.ama

In [9]:
subsample_size = 2000  # for quick demo, try setting to larger values
feature_columns = ['Product_Description', 'Product_Type']
label = 'Sentiment'

train_df = pd.read_csv('product_sentiment_machine_hack/train.csv', index_col=0).sample(2000, random_state=123)
dev_df = pd.read_csv('product_sentiment_machine_hack/dev.csv', index_col=0)
test_df = pd.read_csv('product_sentiment_machine_hack/test.csv', index_col=0)

train_df = train_df[feature_columns + [label]]
dev_df = dev_df[feature_columns + [label]]
test_df = test_df[feature_columns]
print('Number of training samples:', len(train_df))
print('Number of dev samples:', len(dev_df))
print('Number of test samples:', len(test_df))

Number of training samples: 2000
Number of dev samples: 637
Number of test samples: 2728


In [10]:
train_df.head(5)

,Product_Description,Product_Type,Sentiment
4532,they took away the lego pit but replaced it wi...,0,1
1831,#Apple to Open Pop-Up Shop at #SXSW [REPORT]: ...,9,2
3536,RT @mention False Alarm: Google Circles Not Co...,5,1
5157,Will Google reveal a new social network called...,9,2
4643,Niceness RT @mention Less than 2 hours until w...,6,3


In [12]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='Sentiment', path='ag_tabular_product_sentiment_multimodal')
predictor.fit(train_df)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:23:36 PST 2025; root:xnu-11215.81.4~3/RELEASE_ARM64_T8112
CPU Count:          8
Memory Avail:       3.68 GB / 16.00 GB (23.0%)
Disk Space Avail:   56.93 GB / 1862.82 GB (3.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high' 

In [13]:
predictor.leaderboard(dev_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.883830,0.8825,accuracy,0.057359,0.026910,0.263062,0.057359,0.026910,0.263062,1,True,4
1,RandomForestGini,0.883830,0.8875,accuracy,0.098588,0.026416,2.011341,0.098588,0.026416,2.011341,1,True,3
2,WeightedEnsemble_L2,0.883830,0.8875,accuracy,0.101279,0.026628,2.021191,0.002691,0.000212,0.009850,2,True,7
3,ExtraTreesGini,0.871272,0.8575,accuracy,0.044516,0.026974,0.246803,0.044516,0.026974,0.246803,1,True,5
4,ExtraTreesEntr,0.864992,0.8575,accuracy,0.049226,0.026893,0.260282,0.049226,0.026893,0.260282,1,True,6
5,KNeighborsDist,0.723705,0.6875,accuracy,0.047603,0.044602,0.019329,0.047603,0.044602,0.019329,1,True,2
6,KNeighborsUnif,0.720565,0.6800,accuracy,0.054610,0.106696,0.008695,0.054610,0.106696,0.008695,1,True,1


In [16]:
predictor.fit_extra(train_df, presets='best_quality')

ValueError: Unknown `.fit` keyword argument specified: 'presets'
Valid kwargs: ['ag_args', 'ag_args_ensemble', 'ag_args_fit', 'calibrate', 'delay_bag_sets', 'excluded_model_types', 'feature_prune_kwargs', 'hyperparameter_tune_kwargs', 'included_model_types', 'keep_only_best', 'name_suffix', 'num_bag_sets', 'num_stack_levels', 'pseudo_data', 'raise_on_no_models_fitted', 'refit_full', 'save_bag_folds', 'save_space', 'set_best_to_refit_full', 'verbosity']